# 筆記本 5：使用 Google Maps API 獲取地點建議和經緯度
在本教程中，我們將學習如何使用 Google Maps API 從地名查詢中獲取地點建議和其對應的經緯度。我們會使用一個數據庫來避免重複的 API 調用，以提高效率。

## 主要操作步驟
1. 從輸入的地名列表中讀取地點資料。
2. 結合 Google Maps API 查找符合的建議地點及經緯度。
3. 使用本地數據庫儲存搜尋到的地點建議和經緯度，避免重複查詢。
4. 保存結果到指定的 CSV 檔案中。


### 步驟 1：安裝所需的 Python 套件
我們首先需要安裝 `pandas` 和 `googlemaps` 這兩個套件，分別用來處理資料表和執行地點查詢。

In [ ]:
!pip install pandas googlemaps

### 步驟 2：加載數據庫文件
我們將首先嘗試從本地數據庫加載已經經過查詢處理的地名，避免重複查詢。
- 若數據庫文件不存在或為空，會自動建立一個空的資料表。

In [ ]:
import pandas as pd
import os

def load_database_file(database_file_path):
    """
    加載本地數據庫文件，返回包含地名、搜尋關鍵地名、建議地名、緯度和經度的 DataFrame。
    如果數據庫文件不存在或為空，則返回一個空的 DataFrame。
    """
    if os.path.exists(database_file_path):
        try:
            return pd.read_csv(database_file_path, sep='|')
        except pd.errors.EmptyDataError:
            # 如果文件為空，返回一個空的 DataFrame
            return pd.DataFrame(columns=['地名', '搜尋關鍵地名', '建議地名', '緯度', '經度'])
    else:
        # 如果文件不存在，返回一個空的 DataFrame
        return pd.DataFrame(columns=['地名', '搜尋關鍵地名', '建議地名', '緯度', '經度'])

# 測試數據庫加載函數
database_df = load_database_file('database.csv')
database_df.head()

### 步驟 3：處理地名建議和地理位置
接下來，我們將使用 Google Maps API，通過 `autocomplete` 方法獲取地名建議，並進一步通過 `geocode` 方法獲取建議地名的經緯度。
- 每次 API 查詢前，先檢查數據庫，若記錄已存在則跳過查詢；否則執行外部查詢並更新數據庫。


In [ ]:
import googlemaps

def fetch_place_suggestion(gmaps_client, query):
    """
    使用 Google Place Autocomplete API 獲取地點建議，只返回最接近的建議。
    """
    suggestions = gmaps_client.places_autocomplete(query)
    if suggestions:
        return suggestions[0]['description']
    return None


def fetch_geolocation(gmaps_client, place_name):
    """
    根據地名獲取其經緯度，若找不到返回 (None, None)。
    """
    geocode_result = gmaps_client.geocode(place_name)
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
        return (location['lat'], location['lng'])
    return (None, None)

# 測試 Google Maps API 呼叫
gmaps_client = googlemaps.Client(key='YOUR_GOOGLE_API_KEY')
place_suggestion = fetch_place_suggestion(gmaps_client, 'Taipei 101')
# 示例返回的地點建議
print(place_suggestion)

place_geolocation = fetch_geolocation(gmaps_client, place_suggestion)
# 示例返回的地點座標
print(place_geolocation)


### 步驟 4：更新數據庫並存入結果
通過 API 回傳的數據，我們會將其結果更新到現有的數據庫中，然後將最終結果保存到新的 CSV 檔案中。

In [ ]:
def save_database_to_file(database_file_path, database):
    """
    將數據庫內容保存到指定的 CSV 文件，使用 '|' 作為分隔符。
    """
    database.to_csv(database_file_path, index=False, sep='|',
                    columns=['地名', '搜尋關鍵地名', '建議地名', '緯度', '經度'])

# 假設我們取得了新數據，更新數據庫 
new_record = pd.DataFrame({
    '地名': ['新北市'],
    '搜尋關鍵地名': ['Taipei 新北市'],
    '建議地名': ['New Taipei City, Taiwan'],
    '緯度': [25.0169834],
    '經度': [121.4627871]
})
database_df = pd.concat([database_df, new_record], ignore_index=True)

# 將更新後的數據庫保存為 CSV
save_database_to_file('updated_database.csv', database_df)

# 檢查文件是否寫入正確
print('保存完成，請檢查輸出檔案 updated_database.csv')

In [ ]:
!python 05_map_location_coordinates.py -K <YOUR_GOOGLE_MAP_API_KEY> -P 桃園市復興區華陵 -I unique_places.csv -O updated_places.csv -D place_db.csv

### 結論
在本教程中，我們學習到如何通過 Google Maps API 查詢地點相關信息，並結合本地數據庫保存查詢結果。我們也學會了如何避免過多的 API 調用，通過數據庫來有效管理重複地名查詢。